In [1]:
import pandas as pd
import numpy as np
from numpy import linalg as la
import sys
np.set_printoptions(threshold=sys.maxsize)
import datetime

import matplotlib.pyplot as plt
import sklearn

from scipy.linalg import sqrtm

# change the cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:170% !important; }</style>"))

np.set_printoptions(threshold=np.inf)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)


# Import necessary modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

import tensorflow as tf
#load data from data files and place into panda dataframes
#custOrders = pd.read_csv('../FeatureEngineering/data/customer_orders.csv', low_memory=False)

gist_url = "https://gist.githubusercontent.com/GottaGitGoing/c01194dc318f3a31a07d1ae68d31b163/raw/ec37b65f0aa79cf89480981cd9be0df6399dfafb/customer_orders.csv"
custOrders = pd.read_csv(gist_url,low_memory=False)
itemID=pd.read_csv('../SQL/item.csv')
tagLists=pd.read_csv('../SQL/tags.csv')

item_to_tag={} #maps items to tags associated with them
tag_to_item={} #maps tags to the items that have them
ti={}
for index, row in tagLists.iterrows():
    #print(row['item_id'], row['tagName'])
    if row['tagID'] not in ti:
        ti[row['tagID']]=row['tagName']
    
    #create dictionary with key=itemID, value=list of tags that the item has
    if row['item_id'] not in item_to_tag:
        item_to_tag[row['item_id']]=[row['tagName']]
    else:
        item_to_tag[row['item_id']].append(row['tagName'])
        
    #create dictionary with key=tag, value=list of item ID's that have the key-tag
    if row['tagName'] not in tag_to_item:
        tag_to_item[row['tagName']]=[row['item_id']]
    else:
        tag_to_item[row['tagName']].append(row['item_id'])

print(item_to_tag)
print(tag_to_item)
print(itemID)
#turn off panda max column display limit
pd.options.display.max_columns
mns = None



#array holding the tag names in the order they will be in preferences
tagName=['']*len(ti)
for k in ti.keys():
    tagName[k]=ti[k]
#tagNames=['burger', 'sides', 'fatty', 'healthy', 'vegan', 'cold-drink', 'hot-drink', 'dessert']

#
preferences={}
#burger, sides, fatty, healthy, vegan, cold-drink, hot-drink, dessert
preferences[52]=[10.0,10.0,10.0,10.0, 0, 10.0,10.0,10.0]
preferences[194]=[1.5,0,1.8,0, 1 ,0,0,0]
preferences[1]=[0,0,0,0, 1 ,0,0,0]
print(tagName)

{640404923: ['burger', 'fatty'], 640405058: ['burger', 'healthy', 'vegan'], 640405347: ['sides', 'fatty'], 640405331: ['sides', 'fatty'], 640405380: ['sides', 'cold-drink'], 640405395: ['sides', 'cold-drink', 'dessert'], 640405348: ['sides', 'healthy', 'vegan', 'hot-drink']}
{'burger': [640404923, 640405058], 'sides': [640405347, 640405331, 640405380, 640405395, 640405348], 'fatty': [640404923, 640405347, 640405331], 'healthy': [640405058, 640405348], 'vegan': [640405058, 640405348], 'cold-drink': [640405380, 640405395], 'hot-drink': [640405348], 'dessert': [640405395]}
      item_id                       name taste_profile      type
0   640404923                  Hamburger        savory    burger
1   640404963               Cheeseburger        savory    burger
2   640405025         Bacon Cheeseburger        savory    burger
3   640405058              Veggie Burger        savory    burger
4   640405621        Veggie Cheeseburger        savory    burger
5   640405085           Double Ha

In [2]:
#Functions
def makeDF(rows, cols):
    a= np.zeros((rows,cols))    
    return a



#https://towardsdatascience.com/beginners-guide-to-creating-an-svd-recommender-system-1fd7326d1f65

#=====SVD Functions=====
def svd(train, k):
    
    utilMat = np.array(train)
    
    
    # the nan or unavailable entries are masked
    mask = np.isnan(utilMat)
    masked_arr = np.ma.masked_array(utilMat, mask)
    item_means = np.mean(masked_arr, axis=0)
    # nan entries will replaced by the average rating for each item
    utilMat = masked_arr.filled(item_means)
    x = np.tile(item_means, (utilMat.shape[0],1))
    # we remove the per item average from all entries.
    # the above mentioned nan entries will be essentially zero now
    utilMat = utilMat - x
    
    

    # The magic happens here. U and V are user and item features
    U, s, V=np.linalg.svd(utilMat, full_matrices=False)
    s=np.diag(s)
    # we take only the k most significant features
    s=s[0:k,0:k]
    U=U[:,0:k]
    V=V[0:k,:]
    s_root=sqrtm(s)
    Usk=np.dot(U,s_root)
    skV=np.dot(s_root,V)
    UsV = np.dot(Usk, skV)
    UsV = UsV + x
    print("svd done")
    return UsV

#=========

#not used at the moment
def rmse(true, pred):
    # this will be used towards the end
    x = true - pred
    return sum([xi*xi for xi in x])/len(x)

#======

def getPrefArr(user):
    #function gets a userID and returns an array of size(#ofitems on menu)
    #will check each item's tags, then sum the preference-tags score associated with eachitem and return them
    #as an array
    #if no user preferneces, can send back all zeores
    if user not in preferences:
        return np.zeros(len(itemID_Dict))
    
    p=preferences[user]
    #make ratings array with score for each item on the menu
    ret=np.zeros(len(itemID_Dict))
    #iterate through the tags, using tag_to_item dictionary to find itemID's with each tag, using itemList to find the index in ret which 
    # is associated with the itemID, then adding the tag's preference value to that index
    for index, val in enumerate(tagName):
        arr=tag_to_item[val]
        for i in arr:
            ret[itemList.index(i)]+=p[index]  
    
    return ret

#===============

def predict(targetUser):
    #function will return a sorted rankings of menu item using the targetUser order history and prferences (userItem_matrix_df_with_means)
    #if user has no preferences or order history, function will append this user to the userItem_matrix
    global userItem_matrix, userItem_matrix_df_with_means, user_to_row, itemID_Dict, preferences, tagName, binaryTags
    #check if user is already in dataframe, if not then add to all appropriate data structures
    if(targetUser not in user_to_row):
        userItem_matrix=np.append(userItem_matrix, [getPrefArr(targetUser)] , axis=0)        
        user_to_row[targetUser]=userItem_matrix.shape[0]-1
        update_UserItem_dataframe()
        print("added new user")
    
    #call svd function and get predictions
    numOfFeatures=20
    FullOutput=(svd(userItem_matrix_df_with_means, numOfFeatures) )
    output=FullOutput[user_to_row[targetUser]]
    ranking=[]
    #link all ratings to their item ID's
    for index, val in enumerate((list(userItem_matrix_df_with_means))):
        ranking.append((val, output[index]))
        
    #get a new rankings list which has itemID and the scores predicted for it 
    sorted_ranking=[]
    for index, val in enumerate(ranking):
        sorted_ranking.append(( ( val[0], itemID_Dict[val[0]] )  , val[1]))

    #sort list by the score of each item
    sorted_ranking = sorted(sorted_ranking, key=lambda tup: tup[1], reverse=True)

    #check if user has a preference in one of the binary tags, if it does then we will remove all items which do not have that tag
    if targetUser not in preferences:
        p=np.zeros(len(itemID_Dict))
    else:
        p=preferences[targetUser]

    return sorted_ranking



def update_UserItem_dataframe():
    #function commit any changes made in the userItem_matrix to the User-item dataframe(userItem_matrix_df), 
    #then makes a copy and replace all zeroes in the matrix the with mean value of the column(userItem_matrix_df_with_means)
    #use of global keywork allow the changes in the function to be applied to the entire 
    global userItem_matrix, userItem_matrix_df, itemList, userItem_matrix_df_with_means
    userItem_matrix_df=pd.DataFrame(userItem_matrix,columns=itemList)
    data = userItem_matrix_df.copy()
    m = data == 0
    means = np.ma.array(data, mask = m).mean(0)
    userItem_matrix_df_with_means=data + m * means.data

def update_Preferences_from_Orders():
    #wrapper function for taking all rows in orderData_matrix_noPref, updating the user preferences by transforming
    #the item counts into tag values, see getTagsArr() for details
    #NOTE: Also does not erase any orders so calling this function multiple times will just update the user preferences with data that is already represented
    global orderData_matrix_noPref, preferences, row_to_user, orderData_matrix_noPref_df
    #using orderData_matrix_noPref update preference for each user
    for index, row in orderData_matrix_noPref_df.iterrows():
        update_User_Preference(row_to_user[index])


def getTagsArr(itemArr_df):
    #function takes as input a pd.series which contains x cells, where x is the number of items on the menu
    #The function first initializes an array ret, which is of size y, where y is the number of tags that exist
    #Function then iterates through the input and uses the item_to_tag mapping to find the tags each item is associated with,
    #then adds some values to the ret array at the indexes which represent those particular tags
    # ret array is returned as output
    global tagName, item_to_tag
    ret=[0]*len(tagName)
    for itemID in itemArr_df.index:
        if itemID in item_to_tag:
           # print(str(cols)+" : "+str(item_to_tag[cols]))
            for t in item_to_tag[itemID]:
                ind=tagName.index(t)
                ret[ind]+=itemArr_df.loc[itemID]
    return(ret)

def update_User_Preference(userID):
    #function will take a userID, find the row associated with that user in the orderData_matrix_noPref_df matrix, and 
    # use getTagsArr to obtain a item-to-tag mapping
    #then it will add or increment the user's preference scores and return
    global orderData_matrix_noPref, preferences,user_to_row, row_to_user, orderData_matrix_noPref_df, tagName
    #ret will the the array of values that the function returns, it will hold scores for each tag
    #which come from the item-level order counts. It will use the getTagsArr to get these
    #this function can then modify these values(if needed) before adding this
    prefs=[0]*len(tagName)
    if userID not in user_to_row:
        return prefs
    else:   
        prefs=getTagsArr(orderData_matrix_noPref_df.iloc[user_to_row[userID]] )
        #modify scores
        prefs=pd.Series(prefs)*0.1
        

        #if user has existing preferences, add them to the new scores
        if userID in preferences:
            preExisting=preferences[userID]
            prefs=preExisting+prefs
            
        #save prefs to the preferences dictionary, with userID as the key
        #here I cast the preferences from a panda.Series to a python list, but it can remain as a Series if needed
        preferences[userID]=list(prefs)
        return
        

In [3]:
#update_Preferences_from_Orders()

In [4]:

#transform itemID info which is from the csv, into an itemID_Dictionary which will holds an item's index in the menu and its name
# will also create code_to_item, which is another mapping of the itemIndex to the itemName 
itemID['item_score'] = range(1, len(itemID) + 1)
itemID = itemID.drop("item_description", axis=1, errors='ignore')
itemID = itemID.drop("item_image", axis=1, errors='ignore')
#creates dictionary where key is a menu's id number and the value is a 
#tuple with the new menu id embedding(for ML model to predict on) and the menu item's name
itemID_Dict={}
itemList=[]
for i, row in itemID.iterrows():
    itemID_Dict[row.loc["item_id"]]=(row.loc["item_score"]-1, row.loc["name"])
    itemList.append(row.loc["item_id"])

#save item_id to item name
code_to_item={}
for pair in itemID_Dict:
    code_to_item[itemID_Dict[pair][0]]=itemID_Dict[pair][1]
#print out dictionary and itemID dataframe
print("-----------itemID_Dict------------")
print(itemID_Dict)
print("---------code_to_item---------")
print(code_to_item)
print("------itemList--------")
print(itemList)

-----------itemID_Dict------------
{640404923: (0, 'Hamburger'), 640404963: (1, 'Cheeseburger'), 640405025: (2, 'Bacon Cheeseburger'), 640405058: (3, 'Veggie Burger'), 640405621: (4, 'Veggie Cheeseburger'), 640405085: (5, 'Double Hamburger'), 640405112: (6, 'Double Cheeseburger'), 640405172: (7, 'Double Bacon Cheeseburger'), 640405355: (8, 'Small Drink'), 640405371: (9, 'Regular Drink'), 640405380: (10, 'Large Drink'), 640405389: (11, 'Small Shake'), 640405395: (12, 'Regular Shake'), 640405399: (13, 'Large Shake'), 640405296: (14, 'Small Fries'), 640405307: (15, 'Regular Fries'), 640405315: (16, 'Large Fries'), 640405323: (17, 'Small Curly Fries'), 640405331: (18, 'Regular Curly Fries'), 640405339: (19, 'Large Curly Fries'), 640405347: (20, 'Onion Rings'), 640405348: (21, 'Coffee')}
---------code_to_item---------
{0: 'Hamburger', 1: 'Cheeseburger', 2: 'Bacon Cheeseburger', 3: 'Veggie Burger', 4: 'Veggie Cheeseburger', 5: 'Double Hamburger', 6: 'Double Cheeseburger', 7: 'Double Bacon Ch

In [5]:
#generate user-item matrix, and matrix which will populate itself with order data (but not preference data, that will be added later)

#Will also save the user associated with each row in the user-item matrix, the userID is not stored within the 
#user-item matrix and must be saved sepereatly in a user->row dictionary mapping
#We will also reverse the dictionary and create a row->user dictionary and save both mappings

#initalize
userItem_matrix=makeDF(0, len(itemID_Dict))
orderData_matrix_noPref=makeDF(0, len(itemID_Dict))

uniqueUserCount=0
user_to_row={}

#populate user-item matrix
for i, row in custOrders.iterrows():
    currentUser=row.loc["customer_id"]
    currentOrder=row.loc["order_id"]
    currentItemIndex=row.loc["item_id"]
    
    if currentUser not in user_to_row:
        user_to_row[currentUser]=uniqueUserCount
        uniqueUserCount+=1
        
        #also need to add pre-existing preferences
        newUser =  getPrefArr(currentUser) #np.zeros(len(itemID_Dict))
        
        orderData_matrix_noPref=np.append(orderData_matrix_noPref, [newUser_noPref] , axis=0)
        
        newUser_noPref = np.zeros(len(itemID_Dict))
        userItem_matrix= np.append(userItem_matrix, [newUser] , axis=0)

    #for each order on a menu item, increment a cell in the orderData_matrix_noPref and userItem_matrix
    i=user_to_row[currentUser]
    orderData_matrix_noPref[i, itemID_Dict[currentItemIndex][0]-1]+=1
    userItem_matrix[i, itemID_Dict[currentItemIndex][0]-1]+=1


#Will also create a row->user dictionary so we can save both mappings
print(user_to_row)
row_to_user = {value:key for key, value in user_to_row.items()}
print(row_to_user)

NameError: name 'newUser_noPref' is not defined

In [ ]:
#make a dataframe versions of orderData_matrix_noPref and userItem_matrix, with item IDs as the column indexes
orderData_matrix_noPref_df=pd.DataFrame(orderData_matrix_noPref,columns=itemList)
userItem_matrix_df=pd.DataFrame(userItem_matrix,columns=itemList)

In [ ]:
#this segment randomly removed indexes in the userItem_matrix_df. These indexes and the true rating in them would
#have acted like testing data and it would be possible to evaluate the predicted scores better
#but due to time constraints I cannot implement this evalutation metric. As it is not necessary for the functionality of the
#system I have commented out this section

#temp = a copy of userItem_matrix_df which would have some of its scores randomly removed and placed in testingData dictionary
'''
temp=userItem_matrix_df.copy()

randomIndices = np.random.choice(np.arange(temp.size), replace=False,size=int(temp.size * 0.2))
testingData={}
for val in (randomIndices):
#    print(str(val)+" = ["+str(val//len(itemID_Dict))+"]"+ "[ "+str(val%len(itemID_Dict))+"]" )
    row=val//len(itemID_Dict)
    col=val%len(itemID_Dict)
    testingData[val]=temp.iloc[row,col]
    temp.iloc[row,col]=0
    
print(temp)
'''

#print(testingData)
# for val in (randomIndices):
#     print(str(val)+" = ["+str(val//len(itemID_Dict))+"]"+ "[ "+str(val%len(itemID_Dict))+"]" )


In [ ]:
#source: https://stackoverflow.com/questions/60169809/replace-zeros-with-mean-of-non-zeros-along-an-axis-of-array-python-numpy
#this segment replaces the zeroes in the temp dataframe(a copy of the user-item matrix) with the average of the column score
#this non-zerofilled array is temp2

#data=temp #if random values were removed, use this line and comment out next line
data = userItem_matrix_df.copy()
m = data == 0
means = np.ma.array(data, mask = m).mean(0)
userItem_matrix_df_with_means=data + m * means.data

print(userItem_matrix_df_with_means)

In [ ]:
userID_to_predict_on=52
predict(userID_to_predict_on)


In [ ]:
#preferences before
print(preferences)
update_Preferences_from_Orders()
print("===========")
#preferences after
print(preferences)